## Mercado Libre Data

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib as mp
import time
from random import random,randint
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Change the user agent to personal user.
headers = {
         'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
         }

### Example with one link


In [ ]:
#Mercado Libre product link
link = "https://www.mercadolibre.com.mx/bocina-jbl-charge-5-portatil-con-bluetooth-waterproof-pink-110v220v/p/MLM18075342?pdp_filters=category:MLM2868#searchVariation=MLM18075342&position=3&search_layout=stack&type=product&tracking_id=8c0401e0-5a78-41cd-a20f-2932b805b269"
#Request html
page =  requests.get(link, headers=headers)
page.content[0:100]

b'<!doctype html><html><head><meta charset="utf-8"><meta http-equiv="x-ua-compatible" content="ie=edge'

In [ ]:
text_file = open("HTML_19jun/prod1.txt", "w")
n = text_file.write(str(page.content))
text_file.close()

FileNotFoundError: ignored

In [ ]:
#Make sure headers are ok.
page.request.headers

{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [ ]:
#Set up BeautifulSoup format html
soup = BeautifulSoup(page.content, 'html.parser')
#Find de title
title = soup.findAll("h1")[0].text
print(title)

IndexError: ignored

In [ ]:
soup.text

'Amazon.com.mx Algo falló'

In [ ]:
#Find the number of reviews, it should be an intiger
n_reviews = int(soup.findAll("span", {"class":"ui-pdp-review__amount"})[0].text.strip("()"))

print(n_reviews)

IndexError: ignored

In [ ]:
#Find the number of stars
n_stars = float(soup.find("p", {"class":"ui-review-capability__rating__average ui-review-capability__rating__average--desktop"}).text)
print(n_stars)

AttributeError: ignored

In [ ]:
#Find category tree
cat_soup = soup.findAll("ol", {"class":"andes-breadcrumb"})[0].findAll("li")
l_tree = len(cat_soup) #Lenght of category tree
#Save all categories as a list
categories = []
for i in range(l_tree):
  categories.append(cat_soup[i].text)
[categories]

IndexError: ignored

In [ ]:
#Find the available stock
stock = int(re.findall(r'\d+', soup.findAll("span", {"class":"ui-pdp-buybox__quantity__available"})[0].text)[0])
print(stock)

43


In [ ]:
new = soup.findAll("span", {"class":"ui-pdp-subtitle"})[0].text.split("|")[0].strip() == "Nuevo" #Dummy for new product
sales_1 = soup.findAll("span", {"class":"ui-pdp-subtitle"})[0].text.split("|")[1].strip() #Sales category (+100 sold, etc)
print(new, sales_1)

True +100 vendidos


In [ ]:
recommended = soup.findAll("span", {"class":"ui-pdp-promotions-pill-label__text"}) #dummy for recomended product
if len(recommended)>0:
  recommended = recommended[0].text == 'RECOMENDADO'
else:
  recommended = 0
print(recommended)

0


In [ ]:
#Find if they offer free delivery
free_delivery = "Envío gratis a todo el país" in soup.text
print(free_delivery)

True


In [ ]:
#Find seller information
seller_info_soup = soup.findAll("div", {"class":"ui-box-component ui-box-component-pdp__visible--desktop"})[0]

seller_name = seller_info_soup.find("div", {"class":"ui-pdp-seller__header__title"}).text #Name
#seller_link = seller_info_soup.find("a", href = True)["href"] #Link
#seller_sales = seller_info_soup.find("strong").text #Sales category

print(seller_name)#, seller_sales, seller_link)

AttributeError: ignored

In [ ]:
ch_vec_soup = soup.findAll("li", {"class":"ui-pdp-features__item"}) #Vector of characteristics
ch_vec = []
for ch in ch_vec_soup:
  ch_vec.append(ch.text)
ch_vec = [ch_vec]
ch_vec

[[]]

In [ ]:
#Find product benefits or characteristics
product_benefits = soup.find("ul", {"class":"ui-pdp-benefits"}).findAll("a")
n_benefits = len(product_benefits) #Number of benefits
benefits = []
for i in range(n_benefits):
  benefits.append(re.sub("Se abrirá en una nueva ventana","",product_benefits[i].text))
benefits
#Examples: Free returns, insured purchase, Mercado points (Rewards program)

['Compra Protegida']

In [ ]:
#Find current price
curr_price = float(re.sub(",","",soup.find("span", {"class":"andes-money-amount__fraction"}).text))
print(curr_price)

2999.0


### Scrapping Multiple products

In [4]:
def get_ME_data(link,save = 0):
  """
  Function to get Mercado Libre data using a link for a product
  Returns dictionary with 1 observation of title, price, category tree, delivery, among other product info
  """
  data = {}
  data["link"] = link
  #page = requests.get(link,headers = headers)
  time.sleep(random()) #Sleep for random time so that it doesn't get blocked
  page = requests.get(link,headers = headers)
  if save != 0:
    text_file = open(save, "w", encoding = 'latin-1')
    n = text_file.write(str(page.content))
    text_file.close()
  soup = BeautifulSoup(page.content, 'html.parser')
  #print(page.content[0:10])
  if "esta página no existe" in soup.text:
    return 0
  data["title"] = soup.findAll("h1")[0].text
  try:
    data["n_reviews"] = int(soup.find("span", {"class": "ui-pdp-review__amount"}).text.strip("()"))
  except:
    data["n_reviews"] = 0
  try:
    data["n_stars"] = float(soup.find("p", {"class":"ui-review-capability__rating__average ui-review-capability__rating__average--desktop"}).text)
  except:
    data["n_stars"] = 0.0
  cat_soup = soup.findAll("ol", {"class":"andes-breadcrumb"})[0].findAll("li")
  l_tree = len(cat_soup)
  categories = []
  for i in range(l_tree):
    categories.append(cat_soup[i].text)
  categories
  data["l_tree"] = l_tree
  data["categories"] = [categories]

  data["stock"] = int(re.findall(r'\d+', soup.findAll("span", {"class":"ui-pdp-buybox__quantity__available"})[0].text)[0])
  data["new"] = soup.findAll("span", {"class":"ui-pdp-subtitle"})[0].text.split("|")[0].strip() == "Nuevo"
  try:
    data["sales_1"] = soup.findAll("span", {"class":"ui-pdp-subtitle"})[0].text.split("|")[1].strip()
  except:
    data["sales_1"] = soup.find("span", {"class":"ui-pdp-subtitle"}).text
  recommended = soup.findAll("span", {"class":"ui-pdp-promotions-pill-label__text"})
  if len(recommended)>0:
    recommended = recommended[0].text == 'RECOMENDADO'
  else:
    recommended = 0
  data["recommended"] = recommended
  data["free_delivery"] = "Envío gratis a todo el país" in soup.text
  seller_info_soup = soup.findAll("div", {"class":"ui-box-component ui-box-component-pdp__visible--desktop"})[0]
  try:
    data["seller_name"] = seller_info_soup.find("div", {"class":"ui-pdp-seller__header__title"}).text
  except:
    try:
      data["seller_name"] = soup.find("span", {"class":"ui-pdp-color--BLUE ui-pdp-family--REGULAR"}).text
    except:
      data["seller_name"] = ""
  try:
    data["seller_link"] = seller_info_soup.find("a", href = True)["href"]
  except:
    data["seller_link"] = ""
  try:
    data["seller_sales"] = seller_info_soup.find("strong").text
  except:
    data["seller_sales"] = ""
  ch_vec_soup = soup.findAll("li", {"class":"ui-pdp-features__item"}) #Vector of characteristics
  ch_vec = []
  for ch in ch_vec_soup:
    ch_vec.append(ch.text)
  data["ch_vec"] = [ch_vec]
  product_benefits = soup.find("ul", {"class":"ui-pdp-benefits"}).findAll("a")
  n_benefits = len(product_benefits)
  data["n_benefits"] = n_benefits
  benefits = []
  for i in range(n_benefits):
    benefits.append(re.sub("Se abrirá en una nueva ventana","",product_benefits[i].text))
  data["benefits"] = [benefits]
  data["curr_price"] = float(re.sub(",","",soup.find("span", {"class":"andes-money-amount__fraction"}).text))
  data["date"] = 20230727
  return data

In [ ]:
def get_ME_data_saved(link, path):
  """
  Function to get Mercado Libre data using a link for a product
  Returns dictionary with 1 observation of title, price, category tree, delivery, among other product info
  """
  data = {}
  data["link"] = link
  page = open(path,"r", encoding="utf-8").read()
  soup = BeautifulSoup(page, 'html.parser')
  #print(page.content[0:10])
  if "esta página no existe" in soup.text:
    return 0
  data["title"] = soup.findAll("h1")[0].text
  try:
    data["n_reviews"] = int(soup.find("span", {"class": "ui-pdp-review__amount"}).text.strip("()"))
  except:
    data["n_reviews"] = 0
  try:
    data["n_stars"] = float(soup.find("p", {"class":"ui-review-capability__rating__average ui-review-capability__rating__average--desktop"}).text)
  except:
    data["n_stars"] = 0.0
  cat_soup = soup.findAll("ol", {"class":"andes-breadcrumb"})[0].findAll("li")
  l_tree = len(cat_soup)
  categories = []
  for i in range(l_tree):
    categories.append(cat_soup[i].text)
  categories
  data["l_tree"] = l_tree
  data["categories"] = [categories]

  data["stock"] = int(re.findall(r'\d+', soup.findAll("span", {"class":"ui-pdp-buybox__quantity__available"})[0].text)[0])
  data["new"] = soup.findAll("span", {"class":"ui-pdp-subtitle"})[0].text.split("|")[0].strip() == "Nuevo"
  try:
    data["sales_1"] = soup.findAll("span", {"class":"ui-pdp-subtitle"})[0].text.split("|")[1].strip()
  except:
    data["sales_1"] = soup.find("span", {"class":"ui-pdp-subtitle"}).text
  recommended = soup.findAll("span", {"class":"ui-pdp-promotions-pill-label__text"})
  if len(recommended)>0:
    recommended = recommended[0].text == 'RECOMENDADO'
  else:
    recommended = 0
  data["recommended"] = recommended
  data["free_delivery"] = "Envío gratis a todo el país" in soup.text
  seller_info_soup = soup.findAll("div", {"class":"ui-box-component ui-box-component-pdp__visible--desktop"})[0]
  try:
    data["seller_name"] = seller_info_soup.find("div", {"class":"ui-pdp-seller__header__title"}).text
  except:
    try:
      data["seller_name"] = soup.find("span", {"class":"ui-pdp-color--BLUE ui-pdp-family--REGULAR"}).text
    except:
      data["seller_name"] = ""
  try:
    data["seller_link"] = seller_info_soup.find("a", href = True)["href"]
  except:
    data["seller_link"] = ""
  try:
    data["seller_sales"] = seller_info_soup.find("strong").text
  except:
    data["seller_sales"] = ""
  ch_vec_soup = soup.findAll("li", {"class":"ui-pdp-features__item"}) #Vector of characteristics
  ch_vec = []
  for ch in ch_vec_soup:
    ch_vec.append(ch.text)
  data["ch_vec"] = [ch_vec]
  product_benefits = soup.find("ul", {"class":"ui-pdp-benefits"}).findAll("a")
  n_benefits = len(product_benefits)
  data["n_benefits"] = n_benefits
  benefits = []
  for i in range(n_benefits):
    benefits.append(re.sub("Se abrirá en una nueva ventana","",product_benefits[i].text))
  data["benefits"] = [benefits]
  data["curr_price"] = float(re.sub(",","",soup.find("span", {"class":"andes-money-amount__fraction"}).text))
  data["date"] = 20230713
  return data

In [ ]:
#Test to see if it works
link = "https://articulo.mercadolibre.com.mx/MLM-1358005007-mifa-wildbox-wireless-waterproof-bluetooth-speaker-bass-1-_JM#is_advertising=true&position=1&search_layout=stack&type=pad&tracking_id=132d302c-55c4-447d-9701-67fec8c1ff65&is_advertising=true&ad_domain=VQCATCORE_LST&ad_position=1&ad_click_id=NjU0NTM0ODMtZDUwMC00ZTkxLThhNmUtZGQ2NThlN2RmOWFj"
get_ME_data(link)

{'link': 'https://articulo.mercadolibre.com.mx/MLM-1358005007-mifa-wildbox-wireless-waterproof-bluetooth-speaker-bass-1-_JM#is_advertising=true&position=1&search_layout=stack&type=pad&tracking_id=132d302c-55c4-447d-9701-67fec8c1ff65&is_advertising=true&ad_domain=VQCATCORE_LST&ad_position=1&ad_click_id=NjU0NTM0ODMtZDUwMC00ZTkxLThhNmUtZGQ2NThlN2RmOWFj',
 'title': 'Mifa Wildbox Wireless Waterproof Bluetooth Speaker Bass+ 1',
 'n_reviews': 0,
 'n_stars': 0.0,
 'l_tree': 3,
 'categories': [['Electrónica, Audio y Video', 'Audio', 'Bocinas']],
 'stock': 23,
 'new': True,
 'sales_1': '+100 vendidos',
 'recommended': 0,
 'free_delivery': True,
 'seller_name': 'MIFA',
 'seller_link': '',
 'seller_sales': '',
 'ch_vec': [[]],
 'n_benefits': 2,
 'benefits': [['Compra Protegida', 'Mercado Puntos']],
 'curr_price': 2068.0,
 'date': 20230713}

In [5]:
link_search = "https://listado.mercadolibre.com.mx/electronica-audio-video/audio/bocinas/bocina_NoIndex_True#applied_filter_id%3Dcategory%26applied_filter_name%3DCategor%C3%ADas%26applied_filter_order%3D4%26applied_value_id%3DMLM2868%26applied_value_name%3DBocinas%26applied_value_order%3D5%26applied_value_results%3D33592%26is_custom%3Dfalse"
page =  requests.get(link_search, headers=headers)
page.content[0:100]

b'\n<!DOCTYPE html>\n<html lang="es-MX">\n<head><link rel="preconnect" href="https://www.google-analytics'

In [6]:
#Data
links_prods_me = pd.DataFrame({"page":[],"rank":[], "title":[], "link":[]}, index = [])
page_ind = 1
rank = 1
while page_ind <= 10:
  page =  requests.get(link_search, headers=headers)
  text_file = open("/content/drive/MyDrive/Colab Notebooks/data/search_me_"+str(page_ind)+"_12.txt", "w")
  n = text_file.write(str(page.content))
  text_file.close()
  #Set up BeautifulSoup format html
  soup = BeautifulSoup(page.content, 'html.parser')
  Products = soup.findAll("li", {"class":"ui-search-layout__item shops__layout-item"})
  for prod in Products:
    links_prods_me.loc[rank] = [page_ind,rank ,prod.find("a")["title"].strip(), prod.find("a")["href"]]
    rank += 1
  link_search = soup.find("li",{"class":"andes-pagination__button andes-pagination__button--next shops__pagination-button"}).find("a")["href"]
  page_ind+=1
links_prods_me.tail()

,page,rank,title,link
545,10,545,Subwoofer Activo Presonus Air18s 1200w 18 Pulg...,https://www.mercadolibre.com.mx/subwoofer-acti...
546,10,546,Bocina Bluetooth Portátil Mini Led Tws Estéreo...,https://articulo.mercadolibre.com.mx/MLM-16780...
547,10,547,Barra De Som Bluetooth For Tv Par De Barras De...,https://articulo.mercadolibre.com.mx/MLM-23154...
548,10,548,Bocina Bluetooth Ghia Bx91n De 3w Microsd/fm/aux,https://www.mercadolibre.com.mx/bocina-bluetoo...
549,10,549,Bafle Profesional 15 Steelpro Bi-amplificado ...,https://articulo.mercadolibre.com.mx/MLM-13046...


In [7]:
links_prods_me.to_csv("/content/drive/MyDrive/Colab Notebooks/data/links_meli_12")

In [ ]:
data_Me = pd.DataFrame()
for rank, link in zip(links_prods_me["rank"], links_prods_me["link"]):
  try:
    save_path = "/content/drive/MyDrive/Colab Notebooks/data/meli_prods_pages/product_"+str(rank)+"_12.txt"
    data_Me = pd.concat([data_Me, pd.DataFrame(get_ME_data(link,save_path))])
    print("product ", rank, "done")
  except:
    print("product ", rank, "failed")

In [ ]:
links_prods_me = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/links_meli_4").drop('Unnamed: 0', axis = 1)
data_Me = pd.DataFrame()
for rank, link in zip(links_prods_me["rank"], links_prods_me["link"]):
  try:
    save_path = "/content/drive/MyDrive/Colab Notebooks/data/meli_prods_pages/product_"+str(rank)+"_4.txt"
    data_Me = pd.concat([data_Me, pd.DataFrame(get_ME_data_saved(link, save_path))])
    #print("product ", rank, "done")
  except:
    print("product ", rank, "failed")

In [9]:
data_Me = pd.merge(data_Me, links_prods_me, on = "link", how = "inner",suffixes = ("", "_initial"))
data_Me.shape

(489, 22)

In [10]:
data_Me.to_csv("/content/drive/MyDrive/Colab Notebooks/data/data_meli/data_all_12.csv", index= False)
data_Me.head()

,link,title,n_reviews,n_stars,l_tree,categories,stock,new,sales_1,recommended,...,seller_link,seller_sales,ch_vec,n_benefits,benefits,curr_price,date,page,rank,title_initial
0,https://www.mercadolibre.com.mx/bocina-harman-...,Bocina Harman Kardon Onyx Studio 8 portátil co...,0,0.0,3,"[Electrónica, Audio y Video, Audio, Bocinas]",2,True,+500 vendidos,0,...,https://perfil.mercadolibre.com.mx/USF5TECHNOL...,+500,"[Es manos libres., Tipo de bocina: woofer, twe...",1,[Compra Protegida],5883.0,20230727,1,1,Bocina Harman Kardon Onyx Studio 8 portátil co...
1,https://www.mercadolibre.com.mx/bocina-sonos-o...,Bocina Sonos One SL con wifi negra 100V/240V,0,0.0,3,"[Electrónica, Audio y Video, Audio, Bocinas]",6,True,+1000 vendidos,0,...,https://perfil.mercadolibre.com.mx/EUSICA+MX?b...,+500,"[Tipo de bocina: tweeter., Con conectividad wi...",1,[Compra Protegida],4699.0,20230727,1,2,Bocina Sonos One SL con wifi negra 100V/240V
2,https://articulo.mercadolibre.com.mx/MLM-17661...,Bafle Bocina Compatible Bluetooth 8puLG 10w Us...,0,0.0,3,"[Electrónica, Audio y Video, Audio, Bocinas]",41,True,+500 vendidos,0,...,https://perfil.mercadolibre.com.mx/GRUPO+NAVIAR,+5mil,[],1,[Compra Protegida],398.0,20230727,1,4,Bafle Bocina Compatible Bluetooth 8puLG 10w Us...
3,https://www.mercadolibre.com.mx/qsc-altavoz-kl...,Qsc Altavoz Kla12 Altavoz Activo,0,0.0,3,"[Electrónica, Audio y Video, Audio, Bocinas]",15,True,4 vendidos,0,...,,,"[Potencia de 1000W. , Ideal para escuchar tu m...",1,[Compra Protegida],62440.0,20230727,1,5,Qsc Altavoz Kla12 Altavoz Activo
4,https://www.mercadolibre.com.mx/bafle-recargab...,"Bafle Recargable 15pLG Bocina 25,000w Pmpo Ksr...",0,0.0,3,"[Electrónica, Audio y Video, Audio, Bocinas]",6,True,+1000 vendidos,0,...,https://perfil.mercadolibre.com.mx/GRUPO+MARTI...,+10mil,"[Tipo de bocina: woofer., Con conectividad blu...",1,[Compra Protegida],1796.0,20230727,1,6,"Bafle Recargable 15pLG Bocina 25,000w Pmpo Ksr..."


In [ ]:
data_Me.shape

(472, 21)

In [ ]:
expre = []
for ti in data_Me.title:
  expre += re.findall(r'\\x\w\w\\x\w\w', ti)
expre = list(set(expre))

In [ ]:
repr = ['®', 'e', '¼', 'a', 'ñ','o', 'o', 'u', '°', 'i']
def change(x):
  for i in range(len(expre)):
    x = x.replace(expre[i], repr[i])
  return x
data_Me.title = data_Me.title.apply(lambda x: change(x))

#### Now Get the data from MeliPrice (Mercado Libre Price Tracker)

The data contains prices for the past 90 days and the link to buy in Mercado Libre.

Example:

In [ ]:
#Meliprice Example
link = "https://meliprice.com.mx/precio/historico/-roku-express-3930-estandar-full-hd-32mb-negro-con-512mb-de-memoria-ram_1412?days=90"
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
#Find prices
te = soup.findAll("script")[4].text #Body of price plot
start = te.find("data: [")+6 #Find a specific start to a string for the price data
end = te.find("],\n\t")+1  #End of the same string
prices = te[start:end] #Get prices
prices

'[682,699,699,699,699,699,685,685,685,700,689,689,689,689,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669,669]'

In [ ]:
#Find Dates, similar to prices
start_date = te.find("labels: [")+8
end_date = te.find("],\n\n") + 1
dates = te[start_date:end_date]
dates

'["11\\/May","12\\/May","13\\/May","14\\/May","15\\/May","16\\/May","17\\/May","18\\/May","19\\/May","20\\/May","21\\/May","22\\/May","23\\/May","24\\/May","25\\/May","26\\/May","27\\/May","28\\/May","29\\/May","30\\/May","31\\/May","01\\/Jun","02\\/Jun","03\\/Jun","04\\/Jun","05\\/Jun","06\\/Jun","07\\/Jun","08\\/Jun","09\\/Jun","10\\/Jun","11\\/Jun","12\\/Jun","13\\/Jun","14\\/Jun","15\\/Jun","16\\/Jun","17\\/Jun","18\\/Jun","19\\/Jun"]'

In [ ]:
link_me = soup.find("a", { "class":"btn btn-link text-muted pr-0"})["href"] #find mercado libre link
link_me

'https://www.mercadolibre.com.mx/roku-express-3930-estandar-hd-32mb-negro-con-512mb-de-memoria-ram/p/MLM15970894'

In [ ]:
#Test link with the function to get the data from Mercado Libre
get_ME_data(link_me, "example")

{'link': 'https://www.mercadolibre.com.mx/roku-express-3930-estandar-hd-32mb-negro-con-512mb-de-memoria-ram/p/MLM15970894',
 'title': 'Roku Express 3930 estándar HD 32MB negro con 512MB de memoria RAM',
 'n_reviews': 5483,
 'n_stars': 4.8,
 'l_tree': 2,
 'categories': [['Electrónica, Audio y Video', 'Media Streaming']],
 'stock': 673,
 'new': True,
 'sales_1': '+250mil vendidos',
 'recommended': True,
 'free_delivery': True,
 'seller_name': 'Nuvitu Electronics',
 'seller_link': '',
 'seller_sales': '',
 'n_benefits': 2,
 'benefits': [['Compra Protegida', 'Mercado Puntos']],
 'curr_price': 589.0}

#### Get Data from MeliPrice

Now lets do it for a full page on Meliprice, this corresponds to the Electronics, audio, and video category on Mercado Libre.

In [ ]:
link ="https://meliprice.com.mx/celulares-y-telefonia"# "https://meliprice.com.mx/electronica-audio-y-video" #Product list link on Meliprice
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
links_prod = []
for box in soup.findAll("div", {"class":"card-deck mt-2"}):
  for prod in box.findAll("a", href = True):
    links_prod.append(prod["href"]+"?days=50")
len(links_prod) #How may products we will try to get data for, some might not have 50 observations and fail

120

120

In [ ]:
price_dict = {}
date_dict = {}
me_link_dict = {}
#Find prices, dates and links for all products
for link in links_prod:
  page = requests.get(link)
  soup = BeautifulSoup(page.content, 'html.parser')
  try:
    te = soup.findAll("script")[4].text
    start = te.find("data: [")+6
    end = te.find("],\n\t")+1
    prices = te[start:end]
    start_date = te.find("labels: [")+8
    end_date = te.find("],\n\n") + 1
    dates = te[start_date:end_date]
    link_me = soup.find("a", { "class":"btn btn-link text-muted pr-0"})["href"]
    price_dict[link] = prices
    date_dict[link] = dates
    me_link_dict[link] = link_me
  except:
    print(link)

https://meliprice.com.mx/precio/historico/soporte-para-telefono-con-espejo-retrovisor-360º-multiuso_3924?days=50
https://meliprice.com.mx/precio/historico/soporte-para-telefono-con-espejo-retrovisor-360º-multiuso_3924?days=50


In [ ]:
#Remove the link that failed
links_prod.remove('https://meliprice.com.mx/precio/historico/soporte-para-telefono-con-espejo-retrovisor-360º-multiuso_3924?days=50')

In [ ]:
def transform_data(link):
  """
  Function to transform the dictionaries to dataframes
  """
  data = pd.DataFrame({"prices": np.array(price_dict[link].strip("[]").split(",")).astype(float), "dates": date_dict[link].strip("[\"\"]").split("\",\"")})
  data["link_me"] = me_link_dict[link]
  data["link_meliprice"] = link
  return data

In [ ]:
#join dataframes of all products and save dataframe
price_data = transform_data(links_prod[0])
for link in links_prod[1:]:
  price_data = pd.concat([price_data, transform_data(link)])
pd.DataFrame(price_data.link_me.unique()).to_csv("prods_21jun/prod_links_2.csv")
price_data

,prices,dates,link_me,link_meliprice
0,5999.0,13\/May,https://www.mercadolibre.com.mx/xiaomi-redmi-n...,https://meliprice.com.mx/precio/historico/xiao...
1,5999.0,14\/May,https://www.mercadolibre.com.mx/xiaomi-redmi-n...,https://meliprice.com.mx/precio/historico/xiao...
2,5999.0,15\/May,https://www.mercadolibre.com.mx/xiaomi-redmi-n...,https://meliprice.com.mx/precio/historico/xiao...
3,5999.0,16\/May,https://www.mercadolibre.com.mx/xiaomi-redmi-n...,https://meliprice.com.mx/precio/historico/xiao...
4,5999.0,17\/May,https://www.mercadolibre.com.mx/xiaomi-redmi-n...,https://meliprice.com.mx/precio/historico/xiao...
...,...,...,...,...
1,11878.0,17\/Jun,https://articulo.mercadolibre.com.mx/MLM-17604...,https://meliprice.com.mx/precio/historico/appl...
2,11878.0,18\/Jun,https://articulo.mercadolibre.com.mx/MLM-17604...,https://meliprice.com.mx/precio/historico/appl...
3,11878.0,19\/Jun,https://articulo.mercadolibre.com.mx/MLM-17604...,https://meliprice.com.mx/precio/historico/appl...
4,11478.0,20\/Jun,https://articulo.mercadolibre.com.mx/MLM-17604...,https://meliprice.com.mx/precio/historico/appl...


,prices,dates,link_me,link_meliprice
0,5999.0,13\/May,https://www.mercadolibre.com.mx/xiaomi-redmi-n...,https://meliprice.com.mx/precio/historico/xiao...
1,5999.0,14\/May,https://www.mercadolibre.com.mx/xiaomi-redmi-n...,https://meliprice.com.mx/precio/historico/xiao...
2,5999.0,15\/May,https://www.mercadolibre.com.mx/xiaomi-redmi-n...,https://meliprice.com.mx/precio/historico/xiao...
3,5999.0,16\/May,https://www.mercadolibre.com.mx/xiaomi-redmi-n...,https://meliprice.com.mx/precio/historico/xiao...
4,5999.0,17\/May,https://www.mercadolibre.com.mx/xiaomi-redmi-n...,https://meliprice.com.mx/precio/historico/xiao...
...,...,...,...,...
1,11878.0,17\/Jun,https://articulo.mercadolibre.com.mx/MLM-17604...,https://meliprice.com.mx/precio/historico/appl...
2,11878.0,18\/Jun,https://articulo.mercadolibre.com.mx/MLM-17604...,https://meliprice.com.mx/precio/historico/appl...
3,11878.0,19\/Jun,https://articulo.mercadolibre.com.mx/MLM-17604...,https://meliprice.com.mx/precio/historico/appl...
4,11478.0,20\/Jun,https://articulo.mercadolibre.com.mx/MLM-17604...,https://meliprice.com.mx/precio/historico/appl...


In [ ]:
data_Me = pd.DataFrame()
time.sleep(900)
i = 91
for link in links_prod[40:]:
  try:
    data_Me = pd.concat([data_Me, pd.DataFrame(get_ME_data(me_link_dict[link], i))])
    print(i, '. ok')
  except:
    print(link)
  i += 1
data_Me

In [ ]:
data_Me[data_Me.n_reviews>0].shape

(32, 17)

(32, 17)

In [ ]:
data_Me.to_csv("prods_21jun/data_Me91to170.csv")

In [ ]:
data_Me = pd.DataFrame()
for link in links_prod[0:2]:
  data_Me = pd.concat([data_Me, pd.DataFrame(get_ME_data(me_link_dict[link]))])
data_Me

In [ ]:
dir = "/content/drive/MyDrive/Colab Notebooks/HTML_19jun/Prod"
data_Me = pd.DataFrame()
for i in range(1,54):
  try:
    data_Me = pd.concat([data_Me,pd.DataFrame(get_ME_data_saved(dir+str(i)+".html"))])
  except:
    print(i)
data_Me.size

In [ ]:
pd.merge(price_data, me_data, left_on = "link_me", right_on = "link", how = "outer")

### Mercado Libre Price Data comparison

In [ ]:
price_data.columns

In [ ]:
price_data.groupby(by = "link_me").prices.mean().hist()

In [ ]:
price_data.groupby(by = "link_me").prices.std().hist()

In [ ]:
price_data = pd.merge(price_data, price_data.groupby(by = "link_me").prices.mean(), on = "link_me")
price_data = pd.merge(price_data, price_data.groupby(by = "link_me").prices_x.std(), on = "link_me")
price_data

In [ ]:
plt.plot((price_data.prices_y - price_data.prices_x_x)/price_data.prices_x_x)

In [ ]:
plt.hist((price_data.prices_y - price_data.prices_x_x)/price_data.prices_x_x)

In [ ]:
data_Me.n_stars.hist()

In [ ]:
data_Me.n_reviews.hist()

In [ ]:
plt.scatter(data_Me.n_stars, data_Me.n_reviews)

In [ ]:
plt.hist(data_Me.stock)

In [ ]:
plt.scatter(data_Me.curr_price, data_Me.stock)

## Amazon data


In [11]:
#Data
prods_amazon = pd.DataFrame({"page":[],"rank":[], "title":[], "link":[],
                             "n_stars":[], "n_reviews":[], "curr_price":[],
                             "fast_delivery":[], "prime":[], "free_delivery":[], "date":[]}, index = [])
ind_prod = 1
for ind_page in range(1,8):
  path = '/content/drive/MyDrive/Colab Notebooks/data/amazon_prods_pages/page'+str(ind_page)+'_10.html'
  data = {}
  page = open(path,"r").read()
  soup = BeautifulSoup(page, 'html.parser')
  for prod_soup in soup.findAll("div", {"class":"a-section a-spacing-base"}):
    try:
      title = prod_soup.find("span", {"class":"a-size-base-plus a-color-base a-text-normal"}).text
      link_pr = prod_soup.find("a", href = True)["href"]
      try:
        n_stars = float(prod_soup.find("span", {"class":"a-icon-alt"}).text.split()[0])
        n_revs = float(prod_soup.find("span",{"class":"a-size-base s-underline-text"}).text.replace(",", ""))
      except:
        n_stars = 0.0
        n_revs = 0.0
      price = float(prod_soup.find("span",{"class":"a-price-whole"}).text.replace(",", ""))
      prime = (len(prod_soup.findAll("i", {"aria-label":"Amazon Prime"}))==1)
      free_del = ("GRATIS" in prod_soup.text)
      try:
        fast_del = ("mañana" in prod_soup.find("span", {"class":"a-color-base a-text-bold"}).text)
      except:
        fast_del = 0
      prods_amazon.loc[ind_prod] = [ind_page, ind_prod, title, link_pr,
                                    n_stars, n_revs, price, fast_del, prime, free_del, 20230727]
      ind_prod += 1
    except:
      print("product ", ind_prod, " on page ", ind_page, " failed")
      ind_prod += 1
prods_amazon.head()

product  36  on page  1  failed
product  75  on page  2  failed
product  86  on page  2  failed
product  98  on page  2  failed
product  108  on page  2  failed
product  163  on page  3  failed
product  199  on page  4  failed
product  225  on page  4  failed
product  252  on page  4  failed
product  270  on page  5  failed
product  277  on page  5  failed
product  291  on page  5  failed
product  334  on page  6  failed
product  366  on page  6  failed
product  394  on page  7  failed


,page,rank,title,link,n_stars,n_reviews,curr_price,fast_delivery,prime,free_delivery,date
1,1,1,JBL,/JBL-Bluetooth-integrada-Resistente-JBLGO3BLUA...,4.8,24269.0,718.0,True,True,True,20230727
2,1,2,"Bluetooth Portátil, Bluetooth 5.0 Altavoz Ina...",/Bluetooth-Port%C3%A1til-Altavoz-Est%C3%A9reo-...,4.5,3493.0,239.0,True,False,True,20230727
3,1,3,Sony,/Sony-inal%C3%A1mbrica-micr%C3%B3fono-Bluetoot...,4.6,1594.0,1099.0,True,True,True,20230727
4,1,4,AEK CYBER,/sspa/click?ie=UTF8&spc=MTo4ODY3NjY5MjYzODY4MT...,4.2,155.0,599.0,0,True,False,20230727
5,1,5,YOMYM,/sspa/click?ie=UTF8&spc=MTo4ODY3NjY5MjYzODY4MT...,4.1,27.0,499.0,0,True,False,20230727


In [13]:
prods_amazon.to_csv('/content/drive/MyDrive/Colab Notebooks/data/data_amazon/amazon_all_10.csv', index= False)